In [1]:
!pip install langchain langchain-community langchain-core faiss-cpu pypdf sentence-transformers transformers==4.52.4

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 82.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 96.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.2.19
    Uninstalling SQLAlchemy-1.2.19:
      Successfully uninstalled SQLAlchemy-1.2.19
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transf

In [2]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import torch
import re
from langchain.document_loaders import PyPDFLoader

In [3]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

def generate_text(prompt, max_length=1000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2026-01-05 19:38:00.798769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767641881.236721      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767641881.354250      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767641882.407557      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767641882.407593      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767641882.407596      55 computation_placer.cc:177] computation placer alr

...

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
##Extract CV

CV_path = "/kaggle/input/cv-file/Eman Yaser Rezk_CV.pdf"
loader = PyPDFLoader(CV_path)
Document = loader.load()

In [15]:
final_text = "\n".join([doc.page_content for doc in Document])
print(final_text)

Eman Yaser Rezk 
emanyaserr2003@gmail.com   |   01096052095    |   Mansoura, Egypt |  LinkedIn  |   GitHub 
 
SUMMARY 
Computer Science graduate with a solid background in artificial intelligence, machine learning, and 
software development. Dedicated to using technical skills to solve problems and create effective 
solutions. Seeking a job opportunity to apply my skills, contribute to innovative projects, and grow within 
a dynamic team. 
 
EDUCATION 
 Bachelor of Computer Science 
 Faculty of Computer and Information Sciences, Mansoura University 
• GPA score: 4 out of 4, Excellent with honors, Overall percentage: 93.56%. 
 
TRAINING 
Trainee| National Telecommunication Institute (NTI)  
• Python programming basics.  
• Artificial Intelligence foundations.  
• Mathematical Foundations for Deep Learning. 
• Introduction to Deep Learning & TensorFlow.  
Trainee| Information Technology Institute (ITI)  
• Python, NumPy, Probability, Statistics, and Linear Algebra for ML. 
• Data Prepara

In [10]:
name_schema = ResponseSchema(
    name = "full_name",
    description = "Full name of the Candidate as mentioned in the documnet"
)

email_schema = ResponseSchema(
    name = "email",
    description = "The Candidate's email in the document"
)

education_schema = ResponseSchema(
    name = "Education",
    description = "list of information about candidate education, the earned degree, the institution, and graduation year"
)

skills_schema = ResponseSchema(
    name = "Skills",
    description = "list of soft and technical skills mentioned in the candidate CV"
)

experience_schema = ResponseSchema(
    name = "Experience",
    description = "list of candidate's experiences: what is his role, where does he work, and for how many years"
)

response_schema = [name_schema,email_schema,education_schema,skills_schema,experience_schema]

In [12]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()

In [13]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"full_name": string  // Full name of the Candidate as mentioned in the documnet
	"email": string  // The Candidate's email in the document
	"Education": string  // list of information about candidate education, the earned degree, the institution, and graduation year
	"Skills": string  // list of soft and technical skills mentioned in the candidate CV
	"Experience": string  // list of candidate's experiences: what is his role, where does he work, and for how many years
}
```


In [19]:
type(format_instructions)

str

In [14]:
purchase_extraction_template = """
You are a smart assistant that extracts Candidate information from the CV sent to the HR.

Extract the candidate name, email, education, skills and previous experience.

Respond ONLY in JSON format as follows:
{format_instructions}

Example Input:
'''
John Smith – john.smith@email.com
Education: B.Sc. Computer Science, MIT, 2020
Skills: Python, Machine Learning, Data Analysis
Experience:
- Software Engineer at Google (2020–2023)
- Data Scientist at OpenAI (2023–Present)
'''

Now extract from the following input:
"{user_input}"
"""

In [16]:
user_input = final_text

prompt = PromptTemplate(
    template=purchase_extraction_template,
    input_variables=["user_input", "format_instructions"]
).format(user_input=user_input, format_instructions=format_instructions)

response = generate_text(prompt, max_length=3000)[0]
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a smart assistant that extracts Candidate information from the CV sent to the HR.

Extract the candidate name, email, education, skills and previous experience.

Respond ONLY in JSON format as follows:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"full_name": string  // Full name of the Candidate as mentioned in the documnet
	"email": string  // The Candidate's email in the document
	"Education": string  // list of information about candidate education, the earned degree, the institution, and graduation year
	"Skills": string  // list of soft and technical skills mentioned in the candidate CV
	"Experience": string  // list of candidate's experiences: what is his role, where does he work, and for how many years
}
```

Example Input:
'''
John Smith – john.smith@email.com
Education: B.Sc. Computer Science, MIT, 2020
Skills: Python, Machine Learning, Data Analysis
Experience:
- S

In [17]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[-1]}\n```"

json_text = extract_json_block(response)

In [18]:
output_data = output_parser.parse(json_text)
print(output_data)

{'full_name': 'Eman Yaser Rezk', 'email': 'emanyaserr2003@gmail.com', 'Education': 'Bachelor of Computer Science, Faculty of Computer and Information Sciences, Mansoura University', 'Skills': 'Python, C#, HTML, CSS, Dart, Pandas, NumPy, Matplotlib, Seaborn, TensorFlow, Keras, Leadership, Teamwork, Research, Presentation Skills', 'Experience': 'Trainee at National Telecommunication Institute (NTI), Trainee at Information Technology Institute (ITI)'}


In [20]:
arabic_path = "/kaggle/input/arabic-cv/Eman Yaser Arabic CV.pdf"
loader = PyPDFLoader(arabic_path)
Document_arabic = loader.load()
arabic_text = "\n".join([doc.page_content for doc in Document_arabic])
print(arabic_text)

ﻣﺼﺮ  ، ﺼﻮرە اﻟﻤ   :  ﻪ   اﻟﻤﺪ | 01096052095:    اﻟﻬﺎ| emanyaserr2003@gmail.com:  ﯽ ﺮو  ﻟﻜ ٕا    ﺪ ﺮ  اﻟ |
LinkedIn | GitHub
 ﻪ       اﻟ    ﯽ ﻣﻬﺎرا   ﺪام ﺤ   ﺳ    ﻣﻜﺮﺳﻪ.   ﺎٮ    ﺮﻣﺤ اﻟ   ﺮ ﻄﻮ  و    ﻟﯽٓا    ﻌﻠﻢ واﻟ    ﺎﻋﯽ ا  ﺻﻄ   ﻛﺎء اﻟﺪ   ﯽ     ﻪ ﻮ      ﻪ   ﻠ  ﺤ     ﺎٮ اﻟﺤﺎﺳ   ﻋﻠﻮم   ﻪ ﺤ ﺮ  ﺣ
  ﻤﮟ  ﺻ  ﻤﻮ واﻟ    ﻜﺮە   ﻣ   ﻊ ﺎر  ﻣﺴ   ﯽ     واﻟﻤﺴﺎﻫﻤﻪ   ﯽ ﻣﻬﺎرا        ﻄ  ﻟ   ﻋﻤﻞ   ﺮﺻﻪ    ﻋﻠﯽ  ﻟﻠﺤﺼﻮل  ﺳﻌﯽٔا.   ﻌﺎﻟﻪ    ﺣﻠﻮل  ﺎء ﺴ  ٕوا  ﺎﻛﻞ اﻟﻤﺴ  ﻟﺤﻞ
 ﻜﯽ ﺎﻣ    د      ﺮ   .
  ﺎٮ اﻟﺤﺎﺳ   ﻋﻠﻮم   ﯽ    ﻮس ﻜﺎﻟﻮر   
 ﺼﻮرە اﻟﻤ    ﺎﻣﻌﻪ  ﺣ  ، ﻪ ﻋ  ﻣ ٕا    واﻟﻌﻠﻮم   ﺎٮ اﻟﺤﺎﺳ    ﻪ ﻛﻠ 
  ﺮڡ اﻟﺴ   ﻪ   ﻤﺮ      ﺎر ﻣﻤ   ،4  ﻣﮟ  4 :  ﺮاﻛﻤﯽ    ﻣﻌﺪل
93.56 :  ﻪ ﻤﺎﻟ   ﺣٕا     ﻪ ﺴ  اﻟ %
2025 ﻮ ﻮﻟ    –  2021 ﺮ ﻮ  ﻛ ٔا:   ﺮە   اﻟ 
  ﺼﺎ  ٮ ﻟ      ﻮﻣﯽ اﻟ   اﻟﻤﻌﻬﺪ|  ﺪرٮ ﻣ NTI)
 ﻪ ﻠﻌ     ﻪ ﺮﻣﺤ اﻟ    ﺎٮ ﺳﺎﺳ ٔاPython
 ﺎﻋﯽ ا  ﺻﻄ   ﻛﺎء اﻟﺪ   ﺎٮ ﺳﺎﺳ ٔا
    اﻟﻌﻤ   ﻌﻠﻢ ﻟﻠ    ﻪ    ﺎﺻ اﻟﺮ   ﺳﺲٔا  
و     اﻟﻌﻤ   ﻌﻠﻢ اﻟ    ﯽ     ﺪﻣﻪ ﻣ TensorFlow
2024 ﺮ ﻤ    ﺳ  –  ﺴﻄﺲ ﻋٔا:   ﺮە   اﻟ 
  اﻟﻤﻌﻠﻮﻣﺎٮ  ﺎ   ﻮﻟﻮﺣ ﻜ     ﻣﻌﻬﺪ|  ﺪرٮ ﻣ ITI)
Pytho

In [21]:
!pip install arabic-reshaper python-bidi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
import arabic_reshaper
from bidi.algorithm import get_display

def fix_arabic(text):
    reshaped = arabic_reshaper.reshape(text)
    return get_display(reshaped)

fixed_text = fix_arabic(arabic_text)
print(fixed_text)

|ﻟﺍﺮﺪ  ﺍﻜﻟﻭﺮﯽ :emanyaserr2003@gmail.com |ﺎﻬﻟﺍ :01096052095 |ﺪﻤﻟﺍﻪ :  ﻤﻟﺍەﺭﻮﺼ،  ﺮﺼﻣ
LinkedIn | GitHub
ﺣﺮﺤﻪ  ﻡﻮﻠﻋ  ﺳﺎﺤﻟﺍٮﺎ  ﺤﻠﻪ  ﻮﻪ  ﯽ  ﺪﻟﺍﺀﺎﻛ  ﻄﺻﺍﯽﻋﺎ  ﻟﺍﻭﻢﻠﻌ  ﺍﯽﻟ  ﻭﻮﻄﺮ  ﻟﺍﺤﻣﺮ ٮﺎ  .ﻪﺳﺮﻜﻣ  ﺳﺤﻡﺍﺪ  ﺍﺭﺎﻬﻣﯽ  ﻟﺍﻪ
ﻞﺤﻟ  ﺴﻤﻟﺍﻞﻛﺎ  ﺍﻭﺴﺀﺎ  ﻝﻮﻠﺣ  ﻪﻟﺎﻌ  .ﺍﯽﻌﺳ  ﻝﻮﺼﺤﻠﻟ  ﯽﻠﻋ  ﻪﺻﺮ  ﻞﻤﻋ  ﻟﻄ  ﺍﺭﺎﻬﻣﯽ  ﻪﻤﻫﺎﺴﻤﻟﺍﻭ  ﯽ  ﺴﻣﺭﺎﻊ  ﻣەﺮﻜ  ﻟﺍﻭﻮﻤ  ﺻ ﮟﻤ 
.ﺮ  ﺩﻣﺎﯽﻜ
ﺭﻮﻟﺎﻜﺱﻮ  ﯽ  ﻡﻮﻠﻋ  ﺳﺎﺤﻟﺍٮﺎ 
ﻠﻛﻪ  ﺳﺎﺤﻟﺍٮﺎ  ﻡﻮﻠﻌﻟﺍﻭ  ﺍﻣﻋﻪ،  ﺣ ﻪﻌﻣﺎ  ﻤﻟﺍەﺭﻮﺼ
ﻝﺪﻌﻣ  ﯽﻤﻛﺍﺮ : 4  ﮟﻣ 4،  ﻤﻣﺭﺎ  ﺮﻤﻪ  ﺴﻟﺍڡﺮ 
%ﻟﺍﺴﻪ  ﺍﺣ ﻟﺎﻤﻪ : 93.56
ﻟﺍەﺮ  :ﺍﻛﻮﺮ 2021  – ﻟﻮﻮ 2025
(NTIﻣٮﺭﺪ |ﺪﻬﻌﻤﻟﺍ  ﻟﺍﯽﻣﻮ  ﻟٮﺎﺼ 
Pythonﺍﺳﺎﺳٮﺎ  ﻟﺍﺤﻣﺮﻪ  ﻌﻠﻪ
ﺍﺳﺎﺳٮﺎ  ﺪﻟﺍﺀﺎﻛ  ﻄﺻﺍﯽﻋﺎ
ﺍﺲﺳ  ﺮﻟﺍﺻﺎ ﻪ  ﻠﻟﻢﻠﻌ  ﻤﻌﻟﺍ 
TensorFlowﻣﻪﻣﺪ  ﯽ  ﻟﺍﻢﻠﻌ  ﻤﻌﻟﺍ  ﻭ
ﻟﺍەﺮ  :ﺍﻋﺲﻄﺴ  – ﺳﻤﺮ 2024
ITI)ﻣٮﺭﺪ |ﺪﻬﻌﻣ  ﻜﺣﻮﻟﻮﺎ  ٮﺎﻣﻮﻠﻌﻤﻟﺍ 
ﺣﺍﻭٮﺎﻤ  ﺍﻭﺎﺼﺣٮﺎ  ﺤﻟﺍﻭ ﺮ  ﺤﻟﺍﯽﻄ  ﻠﻟﻢﻠﻌ  ﺍﯽﻟ NumPyﻭ Python
ﺼﺤ ﺮ  ﺳﺍﻭﺴﻜڡﺎ  ﻟﺍﺎٮﺎ 
ﻟﺍﻢﻠﻌ  ﺍﯽﻟ  ﺣﻮﻤﻟﺍﻪ  ﻋﻭﺮ  ﺣﻮﻤﻟﺍﻪ
ﻣﻪﻣﺪ  ﯽ  ﻟﺍﻢﻠﻌ  ﻤﻌﻟﺍ 
ﻟﺍەﺮ  :ﻟﻮﻮ  – ﺍﻋﺲﻄﺴ 2024
ﺍﮞﺎﻤ  ﺮﺳﺎ  ﺭﺭٯ 
ﺤﻠﻤﻟﺍﺺ  ﺴﻟﺍﺤﯽﺼ
ﻮﻤﻟﺍٮﻫ  ﻟﺍﻠﻌﻤﻪ
ﻟﺍﺭﺪ  ﺴﻟﺍﻭٮﺍﺩﺎﻬ  ﻬﻤﻟﺍﻪ
ﻟﺍەﺮ :  ﺍﻛﻮﺮ 2024  – ﻮﻮ 2025
Python, TensorFlow, Scikit-learn, Keras, Lang

In [23]:
!pip install python-docx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.6 MB/s eta 0:00:00a 0:00:01


In [24]:
from docx import Document

doc = Document("/kaggle/input/arabic-cv-eman/Eman Yaser Arabic CV.docx")

full_text = []

for para in doc.paragraphs:
    if para.text.strip():
        full_text.append(para.text)

arabic_text = "\n".join(full_text)
print(arabic_text)


إيمان ياسر رزق
البريد الإلكتروني: emanyaserr2003@gmail.com | الهاتف: 01096052095 | المدينة: المنصورة، مصر | LinkedIn | GitHub
الملخص الشخصي
خريجة علوم الحاسبات بخلفية قوية في الذكاء الاصطناعي والتعلم الآلي وتطوير البرمجيات. مكرسة لاستخدام مهاراتي التقنية لحل المشاكل وإنشاء حلول فعالة. أسعى للحصول على فرصة عمل لتطبيق مهاراتي والمساهمة في مشاريع مبتكرة والنمو ضمن فريق ديناميكي.
المؤهلات التعليمية
بكالوريوس في علوم الحاسبات
كلية الحاسبات والعلوم الإعلامية، جامعة المنصورة
معدل تراكمي: 4 من 4، ممتاز بمرتبة الشرف
النسبة الإجمالية: 93.56%
الفترة: أكتوبر 2021 – يوليو 2025
التدريب والشهادات المهنية
متدرب | المعهد القومي للاتصالات (NTI)
أساسيات البرمجة بلغة Python
أساسيات الذكاء الاصطناعي
الأسس الرياضية للتعلم العميق
مقدمة في التعلم العميق و TensorFlow
الفترة: أغسطس – سبتمبر 2024
متدرب | معهد تكنولوجيا المعلومات (ITI)
Python و NumPy والاحتمالات والإحصائيات والجبر الخطي للتعلم الآلي
تحضير واستكشاف البيانات
التعلم الآلي الموجه وغير الموجه
مقدمة في التعلم العميق
الفترة: يوليو – أغسطس 2024
الدورات و

In [48]:
name_schema = ResponseSchema(
    name = "full name",
    description = "اسم المرشح الكامل كما ذكر في النص"
)

email_schema = ResponseSchema(
    name = "email",
    description = "ايميل المرشح في النص"
)

education_schema = ResponseSchema(
    name = "Education",
    description = "قائمة تضم معلومات عن التعليم: المؤهل الدراسي، منشئة التعليم وسنة التخرج"
)

skills_schema = ResponseSchema(
    name = "Skills",
    description = "قائمة من مهارات المرشح سواء التقنية او الشخصية"
)

experience_schema = ResponseSchema(
    name = "Experience",
    description = "قائمة من خبرات المرشح: دور المرشح في الشركة،مكان العمل وعدد سنوات الخبرة"
)

response_schema_arabic = [name_schema,email_schema,education_schema,skills_schema,experience_schema]

In [49]:
output_parser_arabic = StructuredOutputParser.from_response_schemas(response_schema_arabic)
format_instructions_arabic = output_parser_arabic.get_format_instructions()

In [50]:
purchase_extraction_template_arabic = """
أنت مساعد ذكي يستخرج معلومات المرشحين من السيرة الذاتية المُرسلة إلى قسم الموارد البشرية.

استخرج اسم المرشح، وبريده الإلكتروني، ومؤهلاته العلمية، ومهاراته، وخبراته السابقة.

أجب بصيغة JSON فقط كما يلي:
{format_instructions}

مثال على المدخلات:

جون سميث – john.smith@email.com
المؤهلات العلمية: بكالوريوس علوم الحاسوب، معهد ماساتشوستس للتكنولوجيا، 2020
المهارات: بايثون، تعلم الآلة، تحليل البيانات
الخبرات:
- مهندس برمجيات في جوجل (2020–2023)
- عالم بيانات في OpenAI (2023–حتى الآن)

الآن استخرج من المدخلات التالية:
"{user_input}"
"""

In [51]:
user_input = arabic_text

prompt = PromptTemplate(
    template=purchase_extraction_template_arabic,
    input_variables=["user_input", "format_instructions"]
).format(user_input=user_input, format_instructions=format_instructions_arabic)

response = generate_text(prompt, max_length=3000)[0]
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



أنت مساعد ذكي يستخرج معلومات المرشحين من السيرة الذاتية المُرسلة إلى قسم الموارد البشرية.

استخرج اسم المرشح، وبريده الإلكتروني، ومؤهلاته العلمية، ومهاراته، وخبراته السابقة.

أجب بصيغة JSON فقط كما يلي:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"full name": string  // اسم المرشح الكامل كما ذكر في النص
	"email": string  // ايميل المرشح في النص
	"Education": string  // قائمة تضم معلومات عن التعليم: المؤهل الدراسي، منشئة التعليم وسنة التخرج
	"Skills": string  // قائمة من مهارات المرشح سواء التقنية او الشخصية
	"Experience": string  // قائمة من خبرات المرشح: دور المرشح في الشركة،مكان العمل وعدد سنوات الخبرة
}
```

مثال على المدخلات:

جون سميث – john.smith@email.com
المؤهلات العلمية: بكالوريوس علوم الحاسوب، معهد ماساتشوستس للتكنولوجيا، 2020
المهارات: بايثون، تعلم الآلة، تحليل البيانات
الخبرات:
- مهندس برمجيات في جوجل (2020–2023)
- عالم بيانات في OpenAI (2023–حتى الآن)

الآن استخرج من ال

In [52]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[-1]}\n```"

json_text = extract_json_block(response)

In [53]:
output_data = output_parser_arabic.parse(json_text)
print(output_data)

{'full name': 'إيمان ياسر رزق', 'email': 'emanyaserr2003@gmail.com', 'Education': 'بكالوريوس في علوم الحاسبات, كلية الحاسبات والعلوم الإعلامية, جامعة المنصورة, 2021-2025', 'Skills': 'Python, C#, HTML, CSS, Dart, Greedy, Divide and Conquer, Huffman, Sorting algorithms, Tree, List, Dictionary, Heap, Graph, Flutter, Pandas, NumPy, Matplotlib, Seaborn, TensorFlow, Keras, Leading, Teamwork, Research, Presentation', 'Experience': 'متدرب | المعهد القومي للاتصالات (NTI), متدرب | معهد تكنولوجيا المعلومات (ITI), Ancient Aura, GreenGo, HateGuard, Bionic Team, CIS Team MU'}


In [54]:
print(output_data['full name'])

إيمان ياسر رزق
